# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
weather_data_df = pd.read_csv('../WeatherPy/Output/Weather_data.csv')
weather_data_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Norman Wells,CA,65.28,-126.83,57.20,93,75,3.36,07/16/20
1,Rikitea,PF,-23.12,-134.97,73.60,87,100,2.82,07/16/20
2,Mataura,NZ,-46.19,168.86,34.50,94,7,2.39,07/16/20
3,Umm Kaddadah,SD,13.60,26.69,84.27,50,99,13.49,07/16/20
4,Upernavik,GL,72.79,-56.15,42.80,86,95,1.36,07/16/20
...,...,...,...,...,...,...,...,...,...
591,Birin,NP,26.89,88.07,72.10,96,87,3.42,07/16/20
592,Grand Gaube,MU,-20.01,57.66,75.20,64,40,11.41,07/16/20
593,Macapá,BR,0.04,-51.07,78.80,100,40,11.41,07/16/20
594,Selenduma,RU,50.91,106.24,61.81,60,100,6.93,07/16/20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Create a list containing coordinates
coordinates = weather_data_df[['Latitude','Longitude']]

# Create a list of humidity for the weight
humidity = weather_data_df['Humidity']

In [28]:
# Customize the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create a humidity heatmap
fig1 = gmaps.figure(layout=figure_layout, zoom_level=4, center=(43.65,-79.38))

# Create heat map layer
heatmap_layer = gmaps.heatmap_layer(
    coordinates, 
    weights = humidity,
    dissipating=False, 
    max_intensity=humidity.max(),
    point_radius=1.5
)

# Add heat layer
fig1.add_layer(heatmap_layer)

# Display figure
fig1


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# Filter weater_data_df for locations that match criteria A max temperature lower than 80 degrees but higher than 70. 
# Wind speed less than 10 mph. Zero cloudiness.

filter_weather_data = weather_data_df[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp'] < 80) & 
                              (weather_data_df['Wind Speed'] < 10) & (weather_data_df['Cloudiness'] == 0)].reset_index(drop=True)
filter_weather_data
                                  

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Krutikha,RU,53.96,81.21,79.57,48,0,8.23,07/16/20
1,Dovolnoye,RU,54.50,79.67,78.69,51,0,6.06,07/16/20
2,Almaznyy,RU,48.04,40.05,78.80,31,0,4.47,07/16/20
3,La Santisima Trinidad,BO,-14.83,-64.90,70.27,83,0,2.35,07/16/20
4,Juruti,BR,-2.15,-56.09,74.17,83,0,3.91,07/16/20
5,El Escorial,ES,40.59,-4.15,75.20,33,0,3.42,07/16/20
6,Boca do Acre,BR,-8.75,-67.40,71.11,89,0,1.79,07/16/20
7,Mount Isa,AU,-20.73,139.50,75.20,16,0,9.17,07/16/20
8,Semikarakorsk,RU,47.52,40.81,78.80,31,0,4.47,07/16/20
9,Sambek,RU,47.74,39.83,78.80,31,0,4.47,07/16/20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.

In [30]:
# Store into variable named hotel_df
hotel_df = filter_weather_data

# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel'] = ''

* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# base API URL
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'

    # Making the google places request
    response = requests.get(base_url, params=params).json()
    
    # Print which city is currently being searched
    print(f"Processing City {index} | {row['City']}")
        
    try:
        # Print search results
        print(f"The first hotel is {response['results'][0]['name']}")
        
        # Append hotel_df with hotel names for each city (if available)
        hotel_df.loc[index, 'Hotel'] = {response['results'][0]['name']}
    
    except (IndexError):
        # Print error message for no results found
        print('No Hotel in within range. skipping...')
        
        # Append Hotel name as NA if no results found
        hotel_df.loc[index, 'Hotel'] = 'N/A'
    print('-------------------------------------')

Processing City 0 | Krutikha
No Hotel in within range. skipping...
-------------------------------------
Processing City 1 | Dovolnoye
No Hotel in within range. skipping...
-------------------------------------
Processing City 2 | Almaznyy
The first hotel is Gostinitsa Zverevo
-------------------------------------
Processing City 3 | La Santisima Trinidad
The first hotel is Hotel Colonial
-------------------------------------
Processing City 4 | Juruti
The first hotel is Hotel Acauã - ORLA
-------------------------------------
Processing City 5 | El Escorial
The first hotel is Hotel Posada Don Jaime
-------------------------------------
Processing City 6 | Boca do Acre
The first hotel is Hotel Alice
-------------------------------------
Processing City 7 | Mount Isa
The first hotel is ibis Styles Mt Isa Verona
-------------------------------------
Processing City 8 | Semikarakorsk
The first hotel is Отель CONTINENTAL
-------------------------------------
Processing City 9 | Sambek
No H

In [32]:
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel
0,Krutikha,RU,53.96,81.21,79.57,48,0,8.23,07/16/20,N/A
1,Dovolnoye,RU,54.50,79.67,78.69,51,0,6.06,07/16/20,N/A
2,Almaznyy,RU,48.04,40.05,78.80,31,0,4.47,07/16/20,Gostinitsa Zverevo
3,La Santisima Trinidad,BO,-14.83,-64.90,70.27,83,0,2.35,07/16/20,Hotel Colonial
4,Juruti,BR,-2.15,-56.09,74.17,83,0,3.91,07/16/20,Hotel Acauã - ORLA
5,El Escorial,ES,40.59,-4.15,75.20,33,0,3.42,07/16/20,Hotel Posada Don Jaime
6,Boca do Acre,BR,-8.75,-67.40,71.11,89,0,1.79,07/16/20,Hotel Alice
7,Mount Isa,AU,-20.73,139.50,75.20,16,0,9.17,07/16/20,ibis Styles Mt Isa Verona
8,Semikarakorsk,RU,47.52,40.81,78.80,31,0,4.47,07/16/20,Отель CONTINENTAL
9,Sambek,RU,47.74,39.83,78.80,31,0,4.47,07/16/20,N/A


In [33]:
clean_hotel_df = hotel_df.loc[hotel_df['Hotel'] != 'N/A'].reset_index(drop = True)
clean_hotel_df = clean_hotel_df.loc[0:9]
clean_hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel
0,Almaznyy,RU,48.04,40.05,78.80,31,0,4.47,07/16/20,Gostinitsa Zverevo
1,La Santisima Trinidad,BO,-14.83,-64.90,70.27,83,0,2.35,07/16/20,Hotel Colonial
2,Juruti,BR,-2.15,-56.09,74.17,83,0,3.91,07/16/20,Hotel Acauã - ORLA
3,El Escorial,ES,40.59,-4.15,75.20,33,0,3.42,07/16/20,Hotel Posada Don Jaime
4,Boca do Acre,BR,-8.75,-67.40,71.11,89,0,1.79,07/16/20,Hotel Alice
5,Mount Isa,AU,-20.73,139.50,75.20,16,0,9.17,07/16/20,ibis Styles Mt Isa Verona
6,Semikarakorsk,RU,47.52,40.81,78.80,31,0,4.47,07/16/20,Отель CONTINENTAL
7,Hatillo,PR,18.49,-66.83,77.00,93,0,1.99,07/16/20,Hotel Punta Maracayo Resort
8,Novo Aripuanã,BR,-5.12,-60.38,70.88,74,0,0.25,07/16/20,Hotel Gonçalves
9,Pahrump,US,36.21,-115.98,78.01,16,0,3.00,07/16/20,Holiday Inn Express & Suites Pahrump


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [35]:
# Add marker layer ontop of heat map

# Customize the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create a humidity heatmap
fig2 = gmaps.figure(layout=figure_layout, zoom_level=2, center=(43.65,-79.38))

# Create hotel info layer
hotel_info_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)

# Add heat and hotel_info layer
fig2.add_layer(heatmap_layer)
fig2.add_layer(hotel_info_layer)

# Display figure
fig2

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…